In [1]:
# Basic Imports
import pandas as pd
import networkx as nx
import numpy as np
from node2vec import Node2Vec
from gensim.models import KeyedVectors


from os.path import exists

# Import all_gene_disease Dataset

In [2]:
df = pd.read_csv("all_gene_disease_associations.tsv", sep='\t', header=0)
df.head()

,geneId,geneSymbol,DSI,DPI,diseaseId,diseaseName,diseaseType,diseaseClass,diseaseSemanticType,score,EI,YearInitial,YearFinal,NofPmids,NofSnps,source
0,1,A1BG,0.7,0.538,C0001418,Adenocarcinoma,group,C04,Neoplastic Process,0.01,1.0,2008.0,2008.0,1,0,LHGDN
1,1,A1BG,0.7,0.538,C0002736,Amyotrophic Lateral Sclerosis,disease,C18;C10,Disease or Syndrome,0.01,1.0,2008.0,2008.0,1,0,BEFREE
2,1,A1BG,0.7,0.538,C0003578,Apnea,phenotype,C23;C08,Sign or Symptom,0.01,1.0,2017.0,2017.0,1,0,BEFREE
3,1,A1BG,0.7,0.538,C0003864,Arthritis,disease,C05,Disease or Syndrome,0.01,1.0,2019.0,2019.0,1,0,BEFREE
4,1,A1BG,0.7,0.538,C0008373,Cholesteatoma,disease,C17,Disease or Syndrome,0.01,1.0,2020.0,2020.0,1,0,BEFREE


# Count most mentioned diseases
In this way I can decide to focus on some relevant disease

In [3]:
# Show top 20 gene symbols per disease
df.groupby("diseaseName")["geneSymbol"].count()\
.reset_index(name='count')\
.sort_values(['count'], ascending=False)\
.head(20)

,diseaseName,count
20055,Neoplasms,10161
17952,Malignant Neoplasms,8621
23326,Primary malignant neoplasm,8221
18033,Malignant neoplasm of breast,6941
4604,Breast Carcinoma,6776
28471,Tumor Cell Invasion,6626
20027,Neoplasm Metastasis,6385
5519,Carcinogenesis,6243
16738,Liver carcinoma,5725
7057,Colorectal Carcinoma,5473


# Retrieve Information
Now it is necessary to retrieve information about genes for the chosen disease, in my case the **Liver carcinoma**

In [4]:
# Get the gene list in order to label datas
gene_list = df[df["diseaseName"] == "Breast Carcinoma"]["geneSymbol"].drop_duplicates()
gene_list = list(gene_list)

In [5]:
# Display some genes
print(gene_list[:10])

['NAT1', 'NAT2', 'SERPINA3', 'AADAC', 'AAMP', 'AANAT', 'AARS1', 'ABCA1', 'ABCB7', 'ABCF1']


# Import Protein-Protein Interaction Dataset
Import the edge list as a graph

In [6]:
ppi = nx.Graph()
edges = nx.read_edgelist("Biogrid_REDUX.txt")
ppi.add_edges_from(edges.edges())

ppi.remove_edges_from(nx.selfloop_edges(ppi))

In [7]:
print("# Nodes: "+str(len(ppi.nodes)))
print("# Edges: "+str(len(ppi.edges)))

# Nodes: 4263
# Edges: 8048


# Label PPI dataset
Now if a protein of the list is present we put a 1 else a 0 (if necessary)

In [8]:
%%time
if not exists("embeddings.emb"):
    # Precompute probabilities and generate walks - **ON WINDOWS ONLY WORKS WITH workers=1**
    node2vec = Node2Vec(ppi, dimensions=64, walk_length=30, num_walks=200, workers=8)  # Use temp_folder for big graphs
    model = node2vec.fit(window=10, min_count=1, batch_words=4) 
    model.wv.save_word2vec_format("embeddings.emb") # save to file
    wv = model.wv
else:
    wv = KeyedVectors.load_word2vec_format("embeddings.emb")

CPU times: user 129 ms, sys: 3.03 ms, total: 132 ms
Wall time: 131 ms


In [9]:
# Display some data
print(wv["A1BG"])

[ 0.63583446  0.525262   -0.34538925  0.57825655  0.8907158   0.5654882
 -0.26057857  0.6535204   0.5753911   0.6911421   1.2430465  -0.36121282
  1.1483617   0.73211974  0.32190344  0.46187016 -0.10312456  0.3932408
 -0.48767945  0.8974888   0.10346386  0.04698331 -1.1213664   0.2169608
 -0.09109395  0.02253432 -1.3750547  -0.5455049  -0.07808261  0.9353021
  0.42966533  0.04327953 -0.2205511   1.7838341  -1.2173454   0.35714692
 -0.6283102  -0.89529127 -0.33630967  1.4640279   0.31301457 -0.39921093
 -0.09956352  0.87092555  0.17457898  0.49883142  0.7619757  -1.9879476
  0.53596     0.33951506  0.00223186  0.01537907 -0.47599903  0.8532833
  0.48313928  0.29452464  0.20674841 -0.22297807 -0.05739726 -0.90841645
 -1.1654633   0.08074023 -0.23202483 -0.598518  ]


In [10]:
# Retrieve node embeddings and corresponding subjects
node_ids = wv.index_to_key  # list of node IDs
node_embeddings = (
    wv.vectors
)  # numpy.ndarray of size number of nodes times embeddings dimensionality
#node_targets = node_subjects[[int(node_id) for node_id in node_ids]]

In [11]:
# Shape of embeddings
print(node_embeddings.shape)

(4263, 64)


In [25]:
# Label the nodes according to disease chosen
labels = []
for key in ppi.nodes():
    if key in gene_list:
        labels.append(1)
    else:
        labels.append(0)
labels = np.array(labels)
print(labels)
print(len(gene_list))
print("Positive to total ratio: "+"{:.2f}".format( (labels == 1).sum() / len(labels) *100) + "%")

[1 0 0 ... 1 1 1]
6775
Positive to total ratio: 46.19%


# Train machine learing model
With embeddings and labels it is now possible to train a machine learing model. I will try first with **SVM**

In [13]:
# New models imports skl
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import StandardScaler

import torch
from torch.autograd import Variable
from torch_geometric.data import Data
from torch_geometric.data import InMemoryDataset
from torch_geometric.nn import Node2Vec
from torch_geometric.utils.convert import from_networkx

from tqdm.notebook import tqdm

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
%%time
# Node 2 Vec nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'

data = from_networkx(ppi)

model = Node2Vec(data.edge_index, embedding_dim=128, walk_length=20,
             context_size=10, walks_per_node=10,
             num_negative_samples=1, p=1, q=1, sparse=True).to(device)

loader = model.loader(batch_size=128, shuffle=True, num_workers=4)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)
 
def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


@torch.no_grad()
def test():
    model.eval()
    z = model()
    acc = model.test(z[data.train_mask], data.y[data.train_mask],
                     z[data.test_mask], data.y[data.test_mask],
                     max_iter=10)
    return acc


for epoch in tqdm(range(1, 71)):
    loss = train()
    #acc = test()
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')
        

In [ ]:
z = model()

In [ ]:
# from tensor to numpy
node_embeddings = z.detach().cpu().numpy()
print(node_embeddings)

In [ ]:
scaler = StandardScaler()
node_embeddings = scaler.fit_transform(node_embeddings)

In [ ]:
# Rename sets
X = node_embeddings

Y = [1 if node in gene_list else 0 for node in ppi.nodes()]
print("Positive to total ratio: "+"{:.2f}".format( list(Y).count(1) / len(Y) *100) + "%")

In [ ]:
# Dataset split
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=1 - train_ratio, random_state = 42)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state = 42) 

In [ ]:
#clf = svm.SVC(kernel='linear', C=1, random_state=42)
#scores = cross_val_score(clf, X, Y, cv=5)
#print(scores)

In [ ]:
#clf.fit(x_train,y_train)
#clf.score(x_test,y_test)

In [ ]:
clf = MLPClassifier(random_state=1, max_iter=3000).fit(x_train, y_train)

In [ ]:
clf.score(x_test, y_test)

In [ ]:
# Torch
class Feedforward(torch.nn.Module):
    
        def __init__(self, input_size, hidden_size):
            super(Feedforward, self).__init__()
            self.input_size = input_size
            self.hidden_size  = hidden_size
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.relu = torch.nn.ReLU()
            self.fc2 = torch.nn.Linear(self.hidden_size, 1)
            self.sigmoid = torch.nn.Sigmoid()
            
        def forward(self, x):
            hidden = self.fc1(x)
            relu = self.relu(hidden)
            output = self.fc2(relu)
            output = self.sigmoid(F.linear(output))
            return output

In [ ]:
class BinaryClassification(torch.nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        # Number of input features is 12.
        self.layer_1 = torch.nn.Linear(128, 256) 
        self.layer_2 = torch.nn.Linear(256, 256)
        self.layer_out = torch.nn.Linear(256, 1) 
        
        self.relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(p=0.1)
        self.batchnorm1 = torch.nn.BatchNorm1d(256)
        self.batchnorm2 = torch.nn.BatchNorm1d(256)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

In [ ]:
# Training
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)

x_val = torch.FloatTensor(x_val)
y_val = torch.FloatTensor(y_val)

x_test = torch.FloatTensor(x_test)
y_test = torch.FloatTensor(y_test)

In [ ]:
# Nx stuff
adj = nx.to_scipy_sparse_matrix(ppi).tocoo()
row = torch.from_numpy(adj.row.astype(np.int64)).to(torch.long)
col = torch.from_numpy(adj.col.astype(np.int64)).to(torch.long)
edge_index = torch.stack([row,col],dim = 0)

In [ ]:
class P2PDataset(InMemoryDataset):
    def __init__(self,transform = None):
        super(P2PDataset,self).__init__(".",transform,None,None)
        data = Data(edge_index = edge_index)

        data.num_nodes = ppi.number_of_nodes()

        data.x = torch.from_numpy(node_embeddings).type(torch.float32)

        y = torch.from_numpy(labels).type(torch.long)

        data.y = y.clone().detach()

        data.num_classes = 2


        data["x_train"] = x_train
        data["x_test"]  = x_test
        data["x_val"]   = x_val

        self.data, self.slices = self.collate([data])
    

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
x_train = x_train.to(device)
y_train = y_train.to(device)
x_val   = x_val.to(device)
y_val   = y_val.to(device)
x_test  = x_test.to(device)
y_test  = y_test.to(device)

In [ ]:
p2p = P2PDataset()
model = BinaryClassification()


model = model.to(device)

#criterion = torch.nn.BCELoss()
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
model.eval()
y_pred = model(x_test)
before_train = criterion(y_pred.squeeze(), y_test)
print('Test loss before training' , before_train.item())

In [ ]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [ ]:
loss_values = []
val_values = []
epoch = []
epochs = 25
for i in tqdm(range(epochs)):
    print("{:.2f}".format( i / (epochs-1) * 100) + "%",end='\r')
    epoch_acc = 0
    
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
            
            '''
            prediction = model(inputs)
            loss = criterion(prediction.squeeze(), outputs) 
            #print('train loss')
            #print(loss,end='\r')
            loss_values.append(loss.detach())
            optimizer.zero_grad() #zero the parameter gradients
            epoch.append(i)
            loss.backward()       #compute gradients(dloss/dx)
            optimizer.step()      #updates the parameters
            '''
            
            
            optimizer.zero_grad()
            # Forward pass
            y_pred = model(x_train)
            # Compute Loss
            loss = criterion(y_pred.squeeze(), y_train)
            acc = binary_acc(y_pred, y_train.unsqueeze(1))
            epoch_acc += acc.item()

            print('Epoch {}: train loss: {} Acc: {}'.format(epoch, loss.item(),epoch_acc))
            # Backward pass
            loss.backward()
            optimizer.step()
            
            
        elif phase == 'val':
            model.eval()
            
            '''
            prediction_val = model(inputs_val)
            loss_val = criterion(prediction_val.squeeze(), outputs_val) 
            #print('validation loss')
            #print(loss,end='\r')
            val_values.append(loss_val.detach())
            optimizer.zero_grad() #zero the parameter gradients
            '''
            
            # Forward pass
            y_pred = model(x_val)
            # Compute Loss
            loss = criterion(y_pred.squeeze(), y_val)
            print('Epoch {}: validation loss: {}'.format(epoch, loss.item()))
            optimizer.zero_grad()
            

In [ ]:
model.eval()
y_pred = model(x_test)
after_train = criterion(y_pred.squeeze(), y_test) 
print('Test loss after Training' , after_train.item())

In [ ]:
print(binary_acc(y_pred.squeeze(),y_test).item(),"%")

# Stats
Display report after training

In [ ]:
# Stats imports
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [ ]:
y_pred = torch.round(torch.sigmoid(y_pred.squeeze())).cpu().detach().numpy()

In [ ]:
cm = confusion_matrix(y_test.cpu(), y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

In [ ]:
print(classification_report(y_test.cpu(), y_pred))

# GNN model
Now we will proceed to create a GNN model in order to compare the results

In [ ]:
# GNN Imports

# Explainability